In [814]:
import pandas as pd
import numpy as np

df = pd.read_csv(r"../data/class_german_credit.csv", engine="pyarrow")

In [815]:
df

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,67,male,2,own,None,little,1169,6,radio/TV,good
1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,49,male,1,own,little,None,2096,12,education,good
3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,53,male,2,free,little,little,4870,24,car,bad
...,...,...,...,...,...,...,...,...,...,...
995,31,female,1,own,little,None,1736,12,furniture/equipment,good
996,40,male,3,own,little,little,3857,30,car,good
997,38,male,2,own,little,None,804,12,radio/TV,good
998,23,male,2,free,little,little,1845,45,radio/TV,bad


# Pré-processamento dos dados

### Limpeza de dados

#### Remoção de linhas com missing data

In [816]:
df.drop('Checking account',
   axis='columns', inplace=True)

# o numero de NAs dessa coluna é muito alto, perco quase metade do dataset quando mantenho ela
# com essa colunas eu fico em 50% de accuracy

In [817]:
df.dropna(axis=0,inplace= True)

In [818]:
df

,Age,Sex,Job,Housing,Saving accounts,Credit amount,Duration,Purpose,Risk
1,22,female,2,own,little,5951,48,radio/TV,bad
2,49,male,1,own,little,2096,12,education,good
3,45,male,2,free,little,7882,42,furniture/equipment,good
4,53,male,2,free,little,4870,24,car,bad
6,53,male,2,own,quite rich,2835,24,furniture/equipment,good
...,...,...,...,...,...,...,...,...,...
995,31,female,1,own,little,1736,12,furniture/equipment,good
996,40,male,3,own,little,3857,30,car,good
997,38,male,2,own,little,804,12,radio/TV,good
998,23,male,2,free,little,1845,45,radio/TV,bad


#### Remoção de outliers

In [819]:
#aqui eu testei eliminar os outliers baseado no boxplot da analise, não melhorou o resultado

# Q1 = df['Credit amount'].quantile(0.25) # Primeiro quartil
# Q2 = df['Credit amount'].quantile(0.50) # Segundo quartil 
# Q3 = df['Credit amount'].quantile(0.75) # Terceiro quartil
# print(f"Q1: {Q1}")
# print(f"Q2: {Q2}")
# print(f"Q3: {Q3}\n")
# print(df['Credit amount'].median())

# df = df[df['Credit amount'] < Q3 + 1.5 * (Q3-Q1)]
# df

In [820]:
#aqui eu testei eliminar os outliers baseado no boxplot da analise, não melhorou o resultado

# q1 = df['Duration'].quantile(0.25) # primeiro quartil
# q2 = df['Duration'].quantile(0.50) # segundo quartil 
# q3 = df['Duration'].quantile(0.75) # terceiro quartil
# print(f"q1: {q1}")
# print(f"q2: {q2}")
# print(f"q3: {q3}\n")
# print(df['Duration'].median())

# df = df[df['Duration'] < q3 + 1.5 * (q3-q1)]


In [821]:
df = df[df['Job'] != 0]
#resultados melhoram apóes dropar instancias com job '0'

### Normalização e Trasformação

#### Encoding ordinal

In [822]:
#agrupando quite rich e rich como a mesma classe devido ao baixo numero de instancias
#mapeamento = {'little': 0, 'moderate': 1, 'quite rich':2, 'rich':2 }

mapeamento = {'little': 0, 'moderate': 1, 'quite rich':3, 'rich':2 }

df['Saving accounts'] = df['Saving accounts'].map(mapeamento)   

C:\Users\PUC\AppData\Local\Temp\ipykernel_9948\3208206760.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Saving accounts'] = df['Saving accounts'].map(mapeamento)


In [823]:
mapeamento = {'good': 0, 'bad': 1}

df['Risk'] = df['Risk'].map(mapeamento)

C:\Users\PUC\AppData\Local\Temp\ipykernel_9948\2317515701.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Risk'] = df['Risk'].map(mapeamento)


In [824]:
#mapeamento = {'female': 1, 'male': 0}

#df['Sex'] = df['Sex'].map(mapeamento)

#transfomar as variaveis em 1 ou 0 piora o resultado, independente se homem = 0 e mulher = 1 ou homem = 1 e mulher = 0

In [825]:
# mapeamento = {'own': 0, 'rent': 1,'free':2}

# df['Housing'] = df['Housing'].map(mapeamento)

#### One hot encoding

In [826]:
#preciso agrupar os purposes antes de fazer o one hot encoding 
#df['Purpose'] = df['Purpose'].replace({'repairs': 'others', 'domestic appliances': 'others', 'vacation/others': 'others'})

In [827]:
df = pd.get_dummies(df)

In [828]:
df

,Age,Job,Saving accounts,Credit amount,Duration,Risk,Sex_female,Sex_male,Housing_free,Housing_own,Housing_rent,Purpose_business,Purpose_car,Purpose_domestic appliances,Purpose_education,Purpose_furniture/equipment,Purpose_radio/TV,Purpose_repairs,Purpose_vacation/others
1,22,2,0,5951,48,1,True,False,False,True,False,False,False,False,False,False,True,False,False
2,49,1,0,2096,12,0,False,True,False,True,False,False,False,False,True,False,False,False,False
3,45,2,0,7882,42,0,False,True,True,False,False,False,False,False,False,True,False,False,False
4,53,2,0,4870,24,1,False,True,True,False,False,False,True,False,False,False,False,False,False
6,53,2,3,2835,24,0,False,True,False,True,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,31,1,0,1736,12,0,True,False,False,True,False,False,False,False,False,True,False,False,False
996,40,3,0,3857,30,0,False,True,False,True,False,False,True,False,False,False,False,False,False
997,38,2,0,804,12,0,False,True,False,True,False,False,False,False,False,False,True,False,False
998,23,2,0,1845,45,1,False,True,True,False,False,False,False,False,False,False,True,False,False


### Discretização

#### Agrupando categorias

In [829]:
from sklearn.cluster import KMeans

# kmeans = KMeans(n_clusters=5, random_state=42)
# df['cluster'] = kmeans.fit_predict(df[['Credit amount']])
# tentando clusterizar o credit amount

In [830]:
kmeans = KMeans(n_clusters=5, random_state=42)
df['cluster_age'] = kmeans.fit_predict(df[['Age']])
# clusterizar a idade aumenta a taxa de acerto do modelo

In [831]:
#means = KMeans(n_clusters=3, random_state=42)
#df['cluster_duration'] = kmeans.fit_predict(df[['Duration']])
# clusterizar a duração esperada da divida, nao tende a aumetar a taxa de acerto do modelo

In [832]:
df.drop(columns='Age')

,Job,Saving accounts,Credit amount,Duration,Risk,Sex_female,Sex_male,Housing_free,Housing_own,Housing_rent,Purpose_business,Purpose_car,Purpose_domestic appliances,Purpose_education,Purpose_furniture/equipment,Purpose_radio/TV,Purpose_repairs,Purpose_vacation/others,cluster_age
1,2,0,5951,48,1,True,False,False,True,False,False,False,False,False,False,True,False,False,2
2,1,0,2096,12,0,False,True,False,True,False,False,False,False,True,False,False,False,False,3
3,2,0,7882,42,0,False,True,True,False,False,False,False,False,False,True,False,False,False,3
4,2,0,4870,24,1,False,True,True,False,False,False,True,False,False,False,False,False,False,3
6,2,3,2835,24,0,False,True,False,True,False,False,False,False,False,True,False,False,False,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1,0,1736,12,0,True,False,False,True,False,False,False,False,False,True,False,False,False,0
996,3,0,3857,30,0,False,True,False,True,False,False,True,False,False,False,False,False,False,4
997,2,0,804,12,0,False,True,False,True,False,False,False,False,False,False,True,False,False,4
998,2,0,1845,45,1,False,True,True,False,False,False,False,False,False,False,True,False,False,2


### Finalização

In [833]:
X = df.drop(columns=['Risk'])
y = df['Risk']

X.to_csv("../data/df_X.csv", index=False)
y.to_csv('../data/df_y.csv', index=False)
